In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel

logging.getLogger(__name__)

<Logger __main__ (INFO)>

In [7]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
from utils.MIND import MIND

In [3]:
config.reducer = 'bm25'
# config.reducer = 'match'

train_path = config.path + 'MIND/MINDdemo_dev/'
a = MIND(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

In [21]:
dic = pickle.load(open('data/cache/bert/MINDdemo_dev/news_bm25.pkl','rb'))

In [27]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False)
records1 = list(loader1)

In [ ]:
loader2 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=s2)
records2 = list(loader2)

In [29]:
records1[0]

{'impression_index': tensor([1]),
 'user_index': tensor([[1929]]),
 'cdd_id': tensor([[36180, 41328, 41034, 39776, 34983, 37322, 37327, 36307, 36185, 36349]]),
 'his_id': tensor([[ 7499, 34171, 15796, 32731, 32139, 18228,  1736, 30270, 35479, 30833,
          12951, 26650, 30302,  1692, 36210,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 'cdd_encoded_index': tensor([[[  101, 13240, 12134,  2000,  6701, 18466,  1010,  2655, 27056,  9674,
            1005,  1055,  4506,  1005, 21873,  1005,  2867,  2411,  6985,  2037,
           13220,  2043, 15591,  2006,  2019,  5043,  2066,  1996,  2028,  2057,
            2387,  9432,  1010,  2021,  1996,  6234,  4668,  2013, 27224,  1998,
            2500, 16481,  2074,  2129,  2521,  9674,  4625,  1996,  2240,  1012],
